# **12.4 Name_Main_Pattern**

The `if __name__ == '__main__':` pattern is one of the most important Python idioms you'll ever learn. It lets the same `.py` file work both as a **runnable script** and as an **importable module** — without accidentally executing code when another file imports it. Every serious Python developer uses this pattern, and understanding it will save you from confusing bugs.

---

## **The Problem This Solves**

When Python imports a module, it runs **all the top-level code** in that file immediately. If your module has print statements or function calls at the top level, they'll fire every single time the module is imported — even if the importer doesn't want them to.

In [ ]:
# Write a module with problematic top-level code
bad_module = '''
def greet(name):
    return f"Hello, {name}!"

# This runs EVERY time the file is imported!
print("Setting up game...")
print(greet("Ash"))     # Fires on import — annoying!
print("Battle start!")  # Also fires on import!
'''

with open('bad_module.py', 'w') as f:
    f.write(bad_module)

print("Importing bad_module — watch the unwanted output:")
import bad_module  # Surprise! All three prints fire

---

## **Understanding __name__**

Every Python module has a built-in variable called `__name__`. Its value depends on *how* the file is being run:

- When a file is **run directly** (`python my_file.py`), `__name__` is `'__main__'`
- When a file is **imported** by another file, `__name__` is the module's filename (e.g. `'my_file'`)

In [ ]:
# In a notebook, __name__ is '__main__' because we're running cells directly
print(f"This notebook's __name__: {__name__}")

# When a module is imported, its __name__ is its filename
import bad_module
print(f"bad_module's __name__: {bad_module.__name__}")

import math
print(f"math's __name__: {math.__name__}")

# Key insight:
# __name__ == '__main__'  → file is being run directly
# __name__ == 'module_name' → file has been imported

---

## **The if __name__ == '__main__': Pattern**

Wrapping run-only code in this `if` block means it will execute when you run the file directly, but will be **silently skipped** when the file is imported. This is the correct way to structure any Python file that does both jobs.

In [ ]:
# Write a well-structured module using the pattern
good_module = '''
"""pokemon_game.py — A module that is also runnable."""

def greet(name: str) -> str:
    """Return a greeting string for the given trainer name."""
    return f"Hello, {name}!"

def start_battle(pokemon1: str, pokemon2: str) -> str:
    """Return a battle announcement string."""
    return f"{pokemon1} vs {pokemon2} — Battle Start!"

# This block ONLY runs when you execute: python pokemon_game.py
# It is SKIPPED when you do: import pokemon_game
if __name__ == '__main__':
    print(greet("Ash"))
    print(start_battle("Pikachu", "Charizard"))
    print("Game over!")
'''

with open('pokemon_game.py', 'w') as f:
    f.write(good_module)

print("Importing pokemon_game — notice no unwanted output:")
import pokemon_game  # Clean! No prints fire.

print("\nUsing the imported functions:")
print(pokemon_game.greet("Misty"))
print(pokemon_game.start_battle("Starmie", "Onix"))

---

## **Typical Structure of a Python Script**

Professional Python files almost always follow this layout: definitions at the top, entry-point logic guarded by `if __name__ == '__main__':` at the bottom.

In [ ]:
# This shows the standard structure every Python file should follow
template = '''
"""
module_name.py
Brief description of what this file does.
"""

# 1. Imports
import random
import math

# 2. Constants
MAX_LEVEL = 100
STARTERS = ["Bulbasaur", "Charmander", "Squirtle"]

# 3. Functions and classes (reusable, no side effects)
def choose_starter() -> str:
    """Randomly pick a starter Pokemon."""
    return random.choice(STARTERS)

def calculate_damage(power: int, level: int) -> int:
    """Return raw attack damage."""
    return power * level // 10

# 4. Entry point — only runs when executed directly
if __name__ == '__main__':
    starter = choose_starter()
    print(f"Your starter Pokemon is {starter}!")
    print(f"Damage at level 5: {calculate_damage(50, 5)}")
'''

print(template)

---

## **Using a main() Function**

A common refinement is to put all your entry-point logic inside a dedicated `main()` function, then call it from the guard. This makes the entry-point testable and clearly separated from the rest of the code.

In [ ]:
full_script = '''
"""trainer_setup.py — Set up a new Pokemon trainer."""

import random

STARTERS = ["Bulbasaur", "Charmander", "Squirtle"]

def choose_starter() -> str:
    """Randomly pick a starter Pokemon."""
    return random.choice(STARTERS)

def build_profile(trainer: str, starter: str) -> dict:
    """Return a trainer profile dictionary."""
    return {
        "trainer": trainer,
        "starter": starter,
        "badges": 0,
        "team": [starter]
    }

def main():
    """Entry point — runs when the script is executed directly."""
    print("=== New Game ===")
    trainer = "Ash"
    starter = choose_starter()
    profile = build_profile(trainer, starter)
    print(f"Welcome, {profile['trainer']}!")
    print(f"Your starter: {profile['starter']}")
    print(f"Badges: {profile['badges']}")

if __name__ == '__main__':
    main()   # Clean — main() is defined above, called here
'''

with open('trainer_setup.py', 'w') as f:
    f.write(full_script)

# Import without firing main()
import trainer_setup

# Functions are available
starter = trainer_setup.choose_starter()
profile = trainer_setup.build_profile("Misty", starter)
print(f"Imported and used: {profile}")

# You can also call main() explicitly if you want
print("\nCalling main() explicitly:")
trainer_setup.main()

---

## **Why This Matters for Testing**

The pattern also makes automated testing much easier. Test files can import your module and call individual functions without triggering the script's entry-point logic — which might start a game loop, open files, or print things.

In [ ]:
# Imagine this is your test file: test_trainer.py

# Safe to import — main() won't fire
import trainer_setup

# Test individual functions in isolation
def test_build_profile():
    profile = trainer_setup.build_profile("Gary", "Eevee")
    assert profile['trainer'] == "Gary"
    assert profile['starter'] == "Eevee"
    assert profile['badges'] == 0
    print("test_build_profile PASSED")

def test_choose_starter():
    starter = trainer_setup.choose_starter()
    assert starter in trainer_setup.STARTERS
    print("test_choose_starter PASSED")

test_build_profile()
test_choose_starter()

---

## **Practice Exercises**

### **Task 1: Identify __name__**

Print `__name__` from inside this notebook cell and explain what it means.

**Expected Output:**
```
__main__
```

In [ ]:
# Your code here:


### **Task 2: Module's __name__**

Import `pokemon_game` and print its `__name__`.

**Expected Output:**
```
pokemon_game
```

In [ ]:
# Your code here:


### **Task 3: Fix Bad Module**

Rewrite `bad_module.py` so it no longer prints when imported.

**Expected Output when imported:**
```
(no output)
```

In [ ]:
# Your code here:


### **Task 4: Guard Your Script**

Write `encounter.py` with a `random_encounter()` function and a guarded block that calls it.

**Expected Output when imported:**
```
(no output)
```

In [ ]:
# Your code here:


### **Task 5: Add main()**

Write `battle.py` with two helper functions and a `main()` called from the guard.

**Expected Output when main() is called directly:**
```
Pikachu used Thunderbolt!
Damage: 125
```

In [ ]:
# Your code here:


### **Task 6: Import Without Firing**

Import `trainer_setup` and call `choose_starter()` without triggering `main()`.

**Expected Output:**
```
Charmander  (will vary)
```

In [ ]:
# Your code here:


### **Task 7: Conditional Demo**

In a single cell, use `if __name__ == '__main__':` to print a message only when run directly.

**Expected Output:**
```
Running directly!
```

In [ ]:
# Your code here:


### **Task 8: Test a Function**

Import `pokemon_utils` and write a simple assertion test for `is_legendary()`.

**Expected Output:**
```
is_legendary test passed!
```

In [ ]:
import pokemon_utils

# Your test here:


### **Task 9: Full Pattern**

Write `pokedex_lookup.py` with a `lookup(name)` function and a `main()` that demonstrates it, guarded by `__name__`.

**Expected Output when main() called:**
```
Pikachu: Electric type
```

In [ ]:
# Your code here:


### **Task 10: Explain the Pattern**

Fill in the blanks and run the code to see what each `__name__` value is.

**Expected Output:**
```
In notebook: __main__
Imported module: pokemon_utils
```

In [ ]:
import pokemon_utils

# Your code here:


---

## **Summary**

- Every Python file has a `__name__` variable
- When **run directly**: `__name__ == '__main__'`
- When **imported**: `__name__ == 'module_filename'`
- `if __name__ == '__main__':` guards code from running on import
- Put all entry-point logic inside this block (or in `main()`)
- Keep functions and constants outside the guard so they are importable
- Using `main()` makes entry-point code testable

---

## **Quick Reference**

```python
# Standard Python script layout

"""module_name.py — description."""

import something

CONSTANT = 42

def helper_function():
    pass

def main():
    """Entry point."""
    helper_function()

if __name__ == '__main__':
    main()

# Result:
# python script.py   → main() runs
# import script      → main() is skipped
```